## Grabbing the Data
-----

### Setup:
---

In [25]:
# Import dependencies
from pathlib import Path
import numpy as np
import pandas as pd

### Grab NFL Game Data from Past Years:
---

In [26]:
# Set Range of Years
years_list = [2022]
years_list

[2022]

In [60]:
# Set List of Teams
teams_list = ["Arizona Cardinals",
    "Atlanta Falcons",
    "Baltimore Ravens",
    "Buffalo Bills",
    "Carolina Panthers",
    "Chicago Bears",
    "Cincinnati Bengals",
    "Cleveland Browns",
    "Dallas Cowboys",
    "Denver Broncos",
    "Detroit Lions",
    "Green Bay Packers",
    "Houston Texans",
    "Indianapolis Colts",
    "Jacksonville Jaguars",
    "Kansas City Chiefs",
    "Las Vegas Raiders",
    "Los Angeles Chargers",
    "Los Angeles Rams",
    "Miami Dolphins",
    "Minnesota Vikings",
    "New England Patriots",
    "New Orleans Saints",
    "New York Giants",
    "New York Jets",
    "Philadelphia Eagles",
    "Pittsburgh Steelers",
    "San Francisco 49ers",
    "Seattle Seahawks",
    "Tampa Bay Buccaneers",
    "Tennessee Titans",
    "Washington Commanders",
    ]

In [61]:
# Try With a Single Year (2022) First


### Export Past Game Data as CSV Files:
---

In [ ]:
# Export CSV's for Each Team to ../03-Wrangling_Data/Grabbing_Data_Exports
